# THIS NOTEBOOK IS MEANT TO GET USEFUL METRICS ABOUT IMDB DATA

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz 
!tar -xvf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install boto3

In [ ]:
from pyspark.sql import SparkSession, functions as f
import pandas as pd
spark = SparkSession.builder.appName("Movies_Metrics").getOrCreate()


# TABLES SCHEMAS
u.data     --  user id | item id | rating | timestamp. 

u.item     -- movie id | movie title | release date | video release date |

u.user     -- user id | age | gender | occupation | zip code


In [ ]:
data = "https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/u.data"
items = "https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/u.item"
users = "https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/u.user"

In [ ]:
from pandas._libs import index
df_data = pd.read_csv(data, sep="\t", names=["userID", "itemID", "rating", "timestamp"])
df_items = pd.read_csv(items, sep="|", 
                       usecols=range(3), names=["movieID", "movieTitle", "releaseDate"]).astype("str")
df_users= pd.read_csv(users, sep="|", names=["userID", "age", "gender", "occupation", "zipCode"])


In [ ]:
spark_udata = spark.createDataFrame(df_data)
spark_item = spark.createDataFrame(df_items)
spark_users = spark.createDataFrame(df_users)

# START PROJECT 




Finding the three Users who rated more movies

In [ ]:
subdf = spark_udata.groupBy("userID").agg(f.count("*").alias("cuentas"))

In [ ]:
subdf.sort("cuentas", ascending=False).show(3)

Finding the oldest movies dates

In [ ]:
dtf = spark_item.select(f.to_date(spark_item["releaseDate"], "d-MMM-yyyy").alias("fecha"))

In [ ]:
dtf.sort("fecha",ascending=True).filter(dtf.fecha.isNotNull()).show(3)

+----------+
|     fecha|
+----------+
|1922-01-01|
|1926-01-01|
|1930-01-01|
+----------+
only showing top 3 rows



Finding the user details of the user who rated more movies 




In [ ]:
the_user = spark_udata.groupBy("userID").agg(f.count("*").alias("cuentas")).sort("cuentas", ascending=False).limit(1)

In [ ]:
the_user.join(spark_users, "userID").show()

In [ ]:
spark_users.filter(spark_users["userID"]==405).show()

Finding the movie which was most rated

In [ ]:
the_item = spark_udata.groupBy("itemID").agg(f.count("*").alias("cuentas")).sort("cuentas", ascending=False).limit(2)

In [ ]:
the_item.join(spark_item,the_item.itemID == spark_item.movieID).show()

USING THE DISTINCT FX

In [ ]:
#spark_udata.countDistinct()
spark_udata.select(spark_udata.itemID).distinct().count()


# Finding metrics for Gender and occupation

In [ ]:
spark_users.groupBy("gender").count().orderBy("count").show()

The most common occupations among women

In [ ]:
ocp_stats = spark_users.filter(f.lower(spark_users.gender) == 'f').groupBy("occupation").count().orderBy("count", ascending=False)

In [ ]:
ocp_stats.withColumn("perce", ocp_stats.occupation).show()

# THE TOP 10 RATED MOVIES

In [ ]:
joined = spark_udata.join(spark_item, spark_item["movieID"]==spark_udata["itemID"])

In [ ]:
joined.groupBy("movieTitle").count().sort("count", ascending=False).limit(10).show()

# THE BEST RATED MOVIES

In [ ]:
spark_udata.withColumn("rating", 
                       f.when(f.col("rating")==1,"Pesima").when(f.col("rating")==2,"Mala").when(f.col("rating")==3,"Regular").
                       when(f.col("rating")==4,"Buena").when(f.col("rating")==5,"Excelente")).show()

In [ ]:
spark_udata.groupBy("itemID", "rating").count().filter(f.col("itemID")==50).show()

# UDEMY ADVANCE EXAMPLES

In [ ]:
the_movie = spark_udata.groupBy("itemID").agg(f.count("*").alias("ct")).sort(f.col('ct'), ascending=False).limit(1)

In [ ]:
the_movie.join(spark_item, the_movie.itemID == spark_item.movieID).show()


# BROADCAST VARIABLES & UDFs


In [ ]:
most_popular = [{'movieID': 50, 'movieTitle': 'Star Wars (1977)'},{'movieID': 56, 'movieTitle': 'Pulp Fiction (1994)'}, ]

In [ ]:
my_var = spark.sparkContext.broadcast(most_popular)

In [ ]:
results = spark_udata.groupBy('itemID').count().withColumnRenamed('count', 'total').sort('total', ascending=False).limit(20)

In [ ]:
def my_func(idx):
  for i in my_var.value:
    if i['movieID'] == idx:
      return i['movieTitle']
  return None


In [ ]:
lookup = f.udf(my_func)

In [ ]:
results.select('itemID', 'total', lookup(f.col('itemID'))).show()